### 그레디언트 부스팅 계열의 앙상블 모델
- 대표 모델 : XGBoost, LightGBM
- 추가 설치 및 사용법 숙지 필요

- 설치 여부 확인

In [41]:
import pandas as pd
import xgboost, lightgbm

print(f'xgboost : {xgboost.__version__}')
print(f'lightgbm : {lightgbm.__version__}')

xgboost : 1.7.3
lightgbm : 4.1.0


[1] 모듈 로딩 및 데이터 준비

In [42]:
## 데이터 파일 변수 선언
feature_name_file = '../data/human_activity/features.txt'
label_file = '../data/human_activity/activity_labels.txt'

x_train_file = '../data/human_activity/train/X_train.txt'
y_train_file = '../data/human_activity/train/y_train.txt'

x_test_file = '../data/human_activity/test/X_test.txt'
y_test_file = '../data/human_activity/test/y_test.txt'

In [43]:
## 데이터 로딩
import pandas as pd
import numpy as np

feature_nameDF = pd.read_csv(feature_name_file, header=None, names=['col_id', 'col_name'], sep=' ')

In [44]:
# 데이터 확인
feature_nameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    561 non-null    int64 
 1   col_name  561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [45]:
feature_nameDF

,col_id,col_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [46]:
# 중복 체크
feature_nameDF.col_name.duplicated().sum()

84

In [47]:
drop_idx = feature_nameDF[feature_nameDF.col_name.duplicated()].index
drop_idx

Index([316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329,
       330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343,
       395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408,
       409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422,
       474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487,
       488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501],
      dtype='int64')

In [48]:
feature_nameDF.set_index('col_id', inplace=True)
feature_nameDF.drop_duplicates(inplace=True)
feature_nameDF.reset_index(inplace=True)
feature_nameDF

,col_id,col_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
472,557,"angle(tBodyGyroMean,gravityMean)"
473,558,"angle(tBodyGyroJerkMean,gravityMean)"
474,559,"angle(X,gravityMean)"
475,560,"angle(Y,gravityMean)"


In [49]:
feature_nameDF.col_name.duplicated().sum()

0

In [51]:
## => 학습 데이터 로딩
X_trainDF = pd.read_csv(x_train_file, sep='\s+', header=None, usecols=feature_nameDF.col_id.astype('int')-1)

In [52]:
X_trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 477 entries, 0 to 560
dtypes: float64(477)
memory usage: 26.8 MB


In [53]:
# 테스트 데이터 로딩
X_testDF = pd.read_csv(x_test_file, sep='\s+', header=None, usecols=feature_nameDF.col_id.astype('int')-1)
X_testDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 477 entries, 0 to 560
dtypes: float64(477)
memory usage: 10.7 MB
